In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.classification import LogisticRegression

In [2]:
spark = SparkSession.builder.appName('mylogreg').getOrCreate()

In [3]:
df = spark.read.format('libsvm').load('/FileStore/tables/sample_libsvm_data.txt')

In [4]:
df.show(10)

+-----+--------------------+
label| features|
+-----+--------------------+
 0.0|(692,[127,128,129...|
 1.0|(692,[158,159,160...|
 1.0|(692,[124,125,126...|
 1.0|(692,[152,153,154...|
 1.0|(692,[151,152,153...|
 0.0|(692,[129,130,131...|
 1.0|(692,[158,159,160...|
 1.0|(692,[99,100,101,...|
 0.0|(692,[154,155,156...|
 0.0|(692,[127,128,129...|
+-----+--------------------+
only showing top 10 rows

In [5]:
logRegModel = LogisticRegression()

In [6]:
fitted_logReg = logRegModel.fit(df)

In [7]:
log_summary = fitted_logReg.summary

In [8]:
log_summary.predictions.printSchema()

root
-- label: double (nullable = true)
-- features: vector (nullable = true)
-- rawPrediction: vector (nullable = true)
-- probability: vector (nullable = true)
-- prediction: double (nullable = false)

In [9]:
log_summary.predictions.show(5)

+-----+--------------------+--------------------+--------------------+----------+
label| features| rawPrediction| probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
 0.0|(692,[127,128,129...|[19.8534775947478...|[0.99999999761359...| 0.0|
 1.0|(692,[158,159,160...|[-20.377398194908...|[1.41321555111053...| 1.0|
 1.0|(692,[124,125,126...|[-27.401459284891...|[1.25804865126977...| 1.0|
 1.0|(692,[152,153,154...|[-18.862741612668...|[6.42710509170278...| 1.0|
 1.0|(692,[151,152,153...|[-20.483011833009...|[1.27157209200599...| 1.0|
+-----+--------------------+--------------------+--------------------+----------+
only showing top 5 rows

In [10]:
logReg_train,logReg_test = df.randomSplit([0.7,0.3])

In [11]:
final_model = LogisticRegression()

In [12]:
fit_final = final_model.fit(logReg_train)

In [13]:
prediction_and_labels = fit_final.evaluate(logReg_test)

In [14]:
prediction_and_labels.predictions.show(5)

+-----+--------------------+--------------------+--------------------+----------+
label| features| rawPrediction| probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
 0.0|(692,[100,101,102...|[10.0412560153224...|[0.99995643687748...| 0.0|
 0.0|(692,[121,122,123...|[21.9909005694694...|[0.99999999971850...| 0.0|
 0.0|(692,[124,125,126...|[27.9339943382753...|[0.99999999999926...| 0.0|
 0.0|(692,[124,125,126...|[22.7657863625718...|[0.99999999987029...| 0.0|
 0.0|(692,[126,127,128...|[31.7444375082320...|[0.99999999999998...| 0.0|
+-----+--------------------+--------------------+--------------------+----------+
only showing top 5 rows

In [15]:
# to explore the evaluation, use an evaluator object
from pyspark.ml.evaluation import (MulticlassClassificationEvaluator,
                                   BinaryClassificationEvaluator)
#for BinaryClassificationEvaluator, you can get back the area under the ROC curve or the area under the Precision Recall curve
# for MulticlassClassificationEvaluator, you can get back accuracy, precision and recall, etc.

In [16]:
my_eval = BinaryClassificationEvaluator()

In [17]:
my_final_roc = my_eval.evaluate(prediction_and_labels.predictions)

In [18]:
# area under the curve is 1.0 which is perfect...and unrealistic
my_final_roc

Out[ 44 ]: 1.0